## TEST EXAMPLE ON Pyspark


In [1]:
from pyspark import SparkConf, SparkContext

# Reason why we have the getOrCreate code
# http://stackoverflow.com/questions/28999332/how-to-access-sparkcontext-in-pyspark-script
sc = SparkContext.getOrCreate()

In [2]:
#get some basic information of Spark
print(sc.version)
print(sc.appName)
print(sc.defaultParallelism)
print(sc.defaultMinPartitions)

2.2.1
PySparkShell
2
2


In [4]:

from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

from operator import add
import sys

In [4]:
#Create a SparkSession
spark = SparkSession.builder.appName('ReadingData').getOrCreate()

## Load in Data


In [9]:
raw_data = sc.textFile("D:/MusicFile/play/all_play.log.fn")

In [11]:
#set structure type for each col
schema_play = StructType([
    StructField("uid", StringType(), True),
    StructField("device", StringType(), True),
    StructField("song_id", StringType(), True),
    StructField("song_type", StringType(), True),
    StructField("song_name", StringType(), True),
    StructField("singer", StringType(), True),
    StructField("play_time", StringType(), True),
    StructField("song_length", StringType(), True),
    StructField("paid_flag", StringType(), True),
    StructField("date", StringType(), True),
])

schema_no_date = StructType([
    StructField("uid", StringType(), True),
    StructField("device", StringType(), True),
    StructField("song_id", StringType(), True),
    StructField("song_type", StringType(), True),
    StructField("song_name", StringType(), True),
    StructField("singer", StringType(), True),
    StructField("play_time", StringType(), True),
    StructField("song_length", StringType(), True),
    StructField("paid_flag", StringType(), True),
    #StructField("date", StringType(), True),
])


In [27]:
#get the file name
import os
import os.path
date_list = []
def GetFileName(path):
    filename_list = os.listdir(path)
    #that is for getting date of each file
    #for file in filename_list:
    #    date = file.rsplit("_",1)[0]
    #    date_list.append(date)
    return filename_list
playname_list = GetFileName("D:/MusicFile/play")
print(playname_list[0:10],len(playname_list))

['20170301_play.log.fn', '20170302_1_play.log.fn', '20170303_1_play.log.fn', '20170304_1_play.log.fn', '20170305_1_play.log.fn', '20170306_1_play.log.fn', '20170307_1_play.log.fn', '20170308_1_play.log.fn', '20170309_1_play.log.fn', '20170330_3_play.log.fn'] 138


In [28]:
#create an empty dataframe
raw_play = spark.createDataFrame([], schema = schema_play)
#get the date data from each file name
path="D:/MusicFile/play"
for file in playname_list:
    date = file.rsplit('_',1)[0]
    df = spark.read.csv(path = path, sep = "\t", schema = schema_no_date)
    df = df.withColumn("date", lit(date))
    raw_play = raw_play.unionAll(df)
raw_play.createOrReplaceTempView("raw_play")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Tianyi Fang\\AppData\\Local\\Temp\\spark-723f73ca-04fe-475b-919c-3ea2661994f1\\pyspark-711b01b6-37fc-4bfa-820a-258fa02724c0\\tmp65cb88qf'

## Take a look at the raw data

In [29]:
raw_play.show(10)
'''
Notice the first two row contains nothing
'''

+----------+------------------+---------+---------+--------------------+----------------+---------+-----------+---------+--------+
|       uid|            device|  song_id|song_type|           song_name|          singer|play_time|song_length|paid_flag|    date|
+----------+------------------+---------+---------+--------------------+----------------+---------+-----------+---------+--------+
|   264715 | 20170301_play.log|     null|     null|                null|            null|     null|       null|     null|20170301|
|   290416 | 20170301_play.log|     null|     null|                null|            null|     null|       null|     null|20170301|
|154422682 |               ar |20870993 |       1 |                 用情 |          狮子合唱团 |   22013 |       332 |       0 |20170301|
|154421907 |               ip | 6560858 |       0 |             表情不要悲伤 |伯贤&D.O.&张艺兴&朴灿烈 |      96 |       161 |       0 |20170301|
|154422630 |               ar | 3385963 |       1 |Baby, Don't Cry(人...|           

'\nNotice the first two row contains nothing\n'

In [39]:
#Inspect Data
print("DataTypes:",raw_play.dtypes)
print("Return first n rows", raw_play.head(5))
print("Return the nth row", raw_play.take(2))
print("Schema of df", raw_play.schema)


DataTypes: [('uid', 'string'), ('device', 'string'), ('song_id', 'string'), ('song_type', 'string'), ('song_name', 'string'), ('singer', 'string'), ('play_time', 'string'), ('song_length', 'string'), ('paid_flag', 'string'), ('date', 'string')]
Return first n rows [Row(uid='264715 ', device=' 20170301_play.log', song_id=None, song_type=None, song_name=None, singer=None, play_time=None, song_length=None, paid_flag=None, date='20170301'), Row(uid='290416 ', device=' 20170301_play.log', song_id=None, song_type=None, song_name=None, singer=None, play_time=None, song_length=None, paid_flag=None, date='20170301'), Row(uid='154422682 ', device='ar ', song_id='20870993 ', song_type='1 ', song_name='用情 ', singer='狮子合唱团 ', play_time='22013 ', song_length='332 ', paid_flag='0 ', date='20170301'), Row(uid='154421907 ', device='ip ', song_id='6560858 ', song_type='0 ', song_name='表情不要悲伤 ', singer='伯贤&D.O.&张艺兴&朴灿烈 ', play_time='96 ', song_length='161 ', paid_flag='0 ', date='20170301'), Row(uid='154

In [42]:
#Summary Statistics
print("Summary Stats", raw_play.describe().show())

Py4JJavaError: An error occurred while calling o1866.describe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 20.0 failed 1 times, most recent failure: Lost task 1.0 in stage 20.0 (TID 44, localhost, executor driver): java.io.FileNotFoundException: C:\Users\Tianyi Fang\AppData\Local\Temp\blockmgr-428b02d7-b80d-4652-873e-7f65dafaafdd\1c\temp_shuffle_bd1b2cb0-1010-465f-8857-8ee57aaeb88e (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter

In [43]:
#return counts
print("How many records", raw_play.count())
#return distinct counts
print("Distinct records", raw_play.distinct.count())


Py4JJavaError: An error occurred while calling o1866.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 22.0 failed 1 times, most recent failure: Lost task 1.0 in stage 22.0 (TID 47, localhost, executor driver): java.io.FileNotFoundException: C:\Users\Tianyi Fang\AppData\Local\Temp\blockmgr-428b02d7-b80d-4652-873e-7f65dafaafdd\2f\temp_shuffle_b8d4a9c9-62c1-4e76-863a-7baac982d7bd (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.ja

In [45]:
raw_play.columns

['uid',
 'device',
 'song_id',
 'song_type',
 'song_name',
 'singer',
 'play_time',
 'song_length',
 'paid_flag',
 'date']

### Try SQL

In [44]:
from pyspark.sql import functions as F
#use select directly
raw_play.select("uid").show(10)

In [52]:
raw_play.select(raw_play["device"] == "ip").show(5)
#return the logistic/Boolean results

+-------------+
|(device = ip)|
+-------------+
|        false|
|        false|
|        false|
|        false|
|        false|
+-------------+
only showing top 5 rows



In [54]:
#rename...fail
raw_play.select(explode("uid").alias("User_id")).show(5)

NameError: name 'explode' is not defined

In [55]:
#When
raw_play.select("uid", F.when(raw_play.device == "ip",1).otherwise(0)).show(10)


+----------+-----------------------------------------+
|       uid|CASE WHEN (device = ip) THEN 1 ELSE 0 END|
+----------+-----------------------------------------+
|   264715 |                                        0|
|   290416 |                                        0|
|154422682 |                                        0|
|154421907 |                                        0|
|154422630 |                                        0|
|154410267 |                                        0|
|154407793 |                                        0|
|154422626 |                                        0|
|154422681 |                                        0|
|154408091 |                                        0|
+----------+-----------------------------------------+
only showing top 10 rows



In [ ]:
#when
raw_play[raw_play.]